## imports packages for processing data (groups of files) with an atlas 

In [1]:
import sys


# os.environ['R_LIBS_USER'] = '/project/projectdirs/metatlas/r_pkgs/'

# sys.path.insert(0, '/project/projectdirs/metatlas/python_pkgs/')
sys.path.insert(0,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )

from metatlas import metatlas_objects as metob
from metatlas import h5_query as h5q
from metatlas import gui as mgui

import qgrid
import metatlas_get_data_helper_fun as ma_data
import pandas as pd
import os
import tables
import pickle

%matplotlib notebook


:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [2]:
mgui.show_experiments()

/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages/pandas/core/internals.py:1031: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  return self._try_coerce_result(func(values, other))


## use code to filter the groups to the groups you want

In [2]:
#Select the group
# group = metob.retrieve('Groups', name = '%LS_Archetypes%hilic%neg%')
temp_group = metob.retrieve('Groups', name = '%20160202_KBL_QE_HILIC_Brachy_Root_Exudate_POS%',username='*')
# temp_group = metob.retrieve('Groups', name = '%Replicate_Analysis_R2A_HILIC_neg%')
# print len(temp_group)
# print temp_group
group = []
for i,g in enumerate(temp_group):
    if (len(g.items) > 0):# and ('RootCass' not in g.name) and ('_QC_' not in g.name):
        group.append(g)

# group = metob.retrieve('Groups',name='%Avena_Hopland%RootCass%_neg')
# group = metob.retrieve('Groups',name='%Avena_Hopland%RootCass%_pos')
# group = metob.retrieve('Groups',name='%Avena_Hopland%RootExu%')
# group = metob.retrieve('Groups',name='%Avena_Hopland%RootExu%_pos')
# group = metob.retrieve('Groups',name='%Avena_Hopland%RootExu%_neg')

mgui.edit_objects(group)

/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages/pandas/core/internals.py:1031: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  return self._try_coerce_result(func(values, other))


## Put in atlas name here 

In [3]:
#Select the atlas
atlas = metob.retrieve('Atlas',name = '20160229_KBL_Osmolytes_Positive_QE_Hilic',username='*')
# atlas = metob.retrieve('Atlas',name = '%_Positive%_Hilic_QExactive_AvenaExudates_2%')
# atlas = metob.retrieve('Atlas',name = '%_Negative%_Hilic_QExactive_AvenaExudates_2%')
# atlas = metob.retrieve('Atlas',name = '%internal standards%')
# atlas = metob.retrieve('Atlas',name = 'SJ_HM_6550_20160209_Neg_Hilic_Arkin_Ophelia_4RT-Sets',username='*')#_Archetypes_ISTDs%')
# 20151130_LS_Positive_Hilic_QExactive_Archetypes_ISTDs
# 20151130_LS_Negative_Hilic_QExactive_Archetypes_ISTDs
# mgui.edit_objects(atlas)
# len(atlas)
# atlas
from datetime import datetime, date

for i,a in enumerate(atlas):
    print i, a.name,  datetime.utcfromtimestamp(a.last_modified)

0 20160229_KBL_Osmolytes_Positive_QE_Hilic 2016-02-29 22:41:27
1 20160229_KBL_Osmolytes_Positive_QE_Hilic 2016-02-29 23:04:02


## Which atlas do you want?  -1 gets the last one, or typing in the index on the screen can also be used

In [4]:
myAtlas = atlas[-1]

## exports a spreadsheet to your computer that has all the compound info; specify a filename (at the bottom of this script) to name the exported file and where it will be put

In [5]:
# myAtlases = [atlas[0],atlas[1]] #concatenate the atlases you want to use
# myAtlases = [atlas[0]]
compound_list = []
for i in range(len(myAtlas.compound_identifications)):
    if myAtlas.compound_identifications[i].compound:
        compound_list.append(myAtlas.compound_identifications[i].compound[0].name)
    else:
        compound_list.append(myAtlas.compound_identifications[i].name)

cols = ['inchi',
 'mono_isotopic_molecular_weight',
 'creation_time',
 'description',
 'formula',
 'functional_sets',
 'last_modified',
 'reference_xrefs',
 'synonyms',
 'unique_id',
 'url',
 'username']
    
    # print myAtlas[0].compound_identifications[0].compound
atlas_export = pd.DataFrame( index=compound_list, columns=cols)

atlas_export['name'] = compound_list
atlas_export.set_index('name',drop=True)
for i in range(len(myAtlas.compound_identifications)):
    if myAtlas.compound_identifications[i].compound:
        n = myAtlas.compound_identifications[i].compound[0].name
    else:
        n = myAtlas.compound_identifications[i].name
    if myAtlas.compound_identifications[i].compound:
        for c in cols:
                g = getattr(myAtlas.compound_identifications[i].compound[0],c)
                if g:
                    atlas_export.ix[n,c] = getattr(myAtlas.compound_identifications[i].compound[0],c)
    atlas_export.ix[n, 'label'] = myAtlas.compound_identifications[i].name
    atlas_export.ix[n,'rt_min'] = myAtlas.compound_identifications[i].rt_references[0].rt_min
    atlas_export.ix[n,'rt_max'] = myAtlas.compound_identifications[i].rt_references[0].rt_max
    atlas_export.ix[n,'rt_peak'] = myAtlas.compound_identifications[i].rt_references[0].rt_peak
    atlas_export.ix[n,'mz'] = myAtlas.compound_identifications[i].mz_references[0].mz
    atlas_export.ix[n,'mz_tolerance'] = myAtlas.compound_identifications[i].mz_references[0].mz_tolerance
    atlas_export.ix[n,'polarity'] = myAtlas.compound_identifications[i].mz_references[0].detected_polarity
atlas_export.to_csv('Info_for_compounds_in_atlas/20160229_compound_info_osmolytes_pos.csv')


## At the bottom specify a pickle filename

In [7]:
# get and pickle everything This is MSMS, raw MS1 datapoints, compound, group info, and file info
# combine positive and negative mode atlas, by join atlases together vs two compound references
# typically file-groups are NOT split by polarity, files with un-matched polarity will be discarded for analysis of an identification
from metatlas import h5_query as h5q
import os
import tables
import dill
ma_data = reload(ma_data)
data = []
for i,treatment_groups in enumerate(group):
    for j in range(len(treatment_groups.items)):
        myFile = treatment_groups.items[j].hdf5_file
#         try:
#             rt_reference_index = int(treatment_groups.name[-1]) - 1
#         except:
#             rt_reference_index = 3
        print i,len(group),myFile
        row = []
        for compound in myAtlas.compound_identifications:
            result = {}
            result['lcmsrun'] = treatment_groups.items[j]
            result['group'] = treatment_groups
            result['identification'] = compound
            result['data'] = ma_data.get_data_for_a_compound(compound.mz_references[0],
                                    compound.rt_references[0],
                                    [ 'ms1_summary', 'eic', 'msms' ],
                                    myFile,0.2)
            # print result['data']['ms1_summary']
            row.append(result)
        data.append(row)
with open('Pickle_files/20160224_KBL_Brachy_Exudate_Pos_QE_Hilic_osmolytes.pkl','w') as f:
    dill.dump(data,f)

0 5 /global/project/projectdirs/metatlas/raw_data/kblouie/20160202_KBL_QE_HILIC_BrachyExu_Vogel/201622_pHILIC___POS_MSMS_KBL_Qex_Root_BrachyExudate_13_time_3_hr__Run25.h5
0 5 /global/project/projectdirs/metatlas/raw_data/kblouie/20160202_KBL_QE_HILIC_BrachyExu_Vogel/201622_pHILIC___POS_MSMS_KBL_Qex_Root_BrachyExudate_20_time_3_hr__Run34.h5
0 5 /global/project/projectdirs/metatlas/raw_data/kblouie/20160202_KBL_QE_HILIC_BrachyExu_Vogel/201622_pHILIC___POS_MSMS_KBL_Qex_Root_BrachyExudate_21_time_3_hr__Run60.h5
0 5 /global/project/projectdirs/metatlas/raw_data/kblouie/20160202_KBL_QE_HILIC_BrachyExu_Vogel/201622_pHILIC___POS_MSMS_KBL_Qex_Root_BrachyExudate_9_time_3_hr__Run7.h5
1 5 /global/project/projectdirs/metatlas/raw_data/kblouie/20160202_KBL_QE_HILIC_BrachyExu_Vogel/201622_pHILIC___POS_MSMS_KBL_Qex_Root_BrachyExudate_16_time_5_hr__Run31.h5
1 5 /global/project/projectdirs/metatlas/raw_data/kblouie/20160202_KBL_QE_HILIC_BrachyExu_Vogel/201622_pHILIC___POS_MSMS_KBL_Qex_Root_BrachyExudate